In [ ]:
# SVM Optimization on UCI Dataset (e.g., Letter Recognition)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import NuSVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml
import random

# Load dataset (Letter Recognition as example)
df = fetch_openml("letter", version=1, as_frame=True)
X = df.data
y = df.target

# Preprocess
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define hyperparameter space
kernels = ['linear', 'rbf', 'poly']
nu_values = np.linspace(0.1, 0.9, 9)
epsilons = np.linspace(0.01, 0.2, 10)

results = []
best_scores = []
best_convergence = []

for sample_num in range(1, 11):
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, stratify=y, random_state=sample_num)
    best_acc = 0
    best_params = {}
    acc_progress = []

    for i in range(100):
        kernel = random.choice(kernels)
        nu = random.choice(nu_values)
        epsilon = random.choice(epsilons)

        try:
            model = NuSVC(kernel=kernel, nu=nu)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)

            acc_progress.append(acc)

            if acc > best_acc:
                best_acc = acc
                best_params = {"Kernel": kernel, "Nu": nu, "Epsilon": epsilon}
        except:
            acc_progress.append(0)
            continue

    results.append({
        "Sample": f"S{sample_num}",
        "Best Accuracy": round(best_acc * 100, 2),
        "Kernel": best_params.get("Kernel"),
        "Nu": best_params.get("Nu"),
        "Epsilon": best_params.get("Epsilon")
    })
    best_scores.append(best_acc)
    best_convergence.append(acc_progress)

# Save results
results_df = pd.DataFrame(results)
results_df.to_csv("optimized_svm_results.csv", index=False)
print(results_df)

# Plot convergence graph of best
best_index = np.argmax(best_scores)
plt.figure(figsize=(10, 6))
plt.plot(best_convergence[best_index])
plt.xlabel("Iteration")
plt.ylabel("Accuracy")
plt.title(f"Fitness (bestAccuracy) - Sample S{best_index+1}")
plt.grid(True)
plt.savefig("best_convergence_graph.png")
plt.show()
